Question 1

Load the energy data from the file Energy Indicators.xls.
It is a list of indicators of energy supply and renewable electricity production from the United Nations for the year 2013.

It should be put into a DataFrame with the variable name of "energy"

Make sure to exclude the footer and header information from the datafile.

The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

Convert Energy Supply to gigajoules (there are 1,000,000 gigajoules in a petajoule).

For all countries which have missing data (e.g. data with ...) make sure this is reflected as np.NaN values.

Rename the following list of countries (for use in later questions):

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, e.g.:


It is a csv containing countries' GDP from 1960 to 2015 from World Bank. Call this DataFrame "GDP"

Make sure to skip the header, and rename the following list of countries:
. Call this DataFrame "ScimEn"

Join the three datasets: Energy, GDP, and ScimEn into a new dataset (using the intersection of country names). Use only the 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15).

The index of this DataFrame should be the name of the country, and the columns should be

Function "answer_one" should return the resulted DataFrame (20 columns and 15 entries)

In [1]:
import pandas as pd
import numpy as np

def answer_one():
    energy=pd.read_excel('Energy Indicators.xls', skiprows=17,skipfooter=38)
    energy.drop(energy.columns[[0,1]],axis=1,inplace=True)
    energy.columns=['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    energy.replace('...', np.nan,inplace = True)
    energy['Energy Supply'] = energy['Energy Supply']*1000000
    energy['Country'].replace({"Republic of Korea": "South Korea", "United States of America20": "United States", "United Kingdom of Great Britain and Northern Ireland19": "United Kingdom","China, Hong Kong Special Administrative Region3": "Hong Kong"}, inplace=True)
    energy['Country'].replace({'Bolivia (Plurinational State of)':'Bolivia','Switzerland17':'Switzerland'},inplace=True)
    energy['Country'].replace({"Iran (Islamic Republic of)": "Iran", "France6": "France", "Italy9": "Italy", 
                            "Ukraine18": "Ukraine", "Spain16": "Spain", "Serbia15": "Serbia", "Saudi Arabia14": "Saudi Arabia", 
                            "Portugal13": "Portugal", "Netherlands12": "Netherlands", "Kuwait11": "Kuwait", 
                            "Japan10": "Japan", "Denmark5": "Denmark","China2": "China","Australia1": "Australia"}, inplace=True)
    pd.set_option('display.max_rows', energy.shape[0]+1)

    GDP=pd.read_csv('world_bank.csv', skiprows=4)
    GDP['Country Name'].replace({'Korea, Rep.':'South Korea','Iran, Islamic Rep.':'Iran','Hong Kong SAR, China':'Hong Kong'},inplace=True)
    GDP.rename(columns={'Country Name':'Country'},inplace=True)
    pd.set_option('display.max_rows', GDP.shape[0]+1)
    ScimEn=pd.read_excel('scimagojr country rank 1996-2021.xlsx')
    ScimEn=ScimEn[0:15]
    df=pd.merge(ScimEn, energy , how = 'inner' , left_on = 'Country' , right_on = 'Country')
    df_all=pd.merge(df,GDP, how = 'inner' , left_on = 'Country' , right_on = 'Country')
    df_all=df_all.set_index('Country')
    df_all=df_all[['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
    return df_all
answer_one()

FileNotFoundError: [Errno 2] No such file or directory: 'Energy Indicators.xls'

Question 2

What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.

In [4]:
def answer_two():
    Top15=answer_one()
    gdp=Top15[['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
    avg_gdp=gdp.mean(axis=1)

    return avg_gdp
answer_two()

Country
China                 6.927702e+12
United States         1.570403e+13
India                 1.602352e+12
Japan                 5.239642e+12
United Kingdom        2.792011e+12
Germany               3.523342e+12
Russian Federation    1.666746e+12
Canada                1.616359e+12
Italy                 2.142986e+12
South Korea           1.221372e+12
France                2.691337e+12
Iran                  4.563261e+11
Spain                 1.400727e+12
Brazil                1.988889e+12
Australia             1.207106e+12
dtype: float64

Question 3

By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

In [8]:
def answer_three():
    Top15=answer_one()
    avg=answer_two().sort_values(ascending=False)
    country=avg.index[5]
    value=Top15.loc[country]['2015']-Top15.loc[country]['2006']
    return value
answer_three()

118652421857.7998

Question 4

Create a new column that is the ratio of Self-Citations to Total Citations. What is the maximum value for this new column, and what country has the highest ratio?

In [33]:
def answer_four():
    Top15=answer_one()
    ratio=Top15['Self-citations']/Top15['Citations']
    Top15=Top15.assign(Ratio=ratio)
    ans=ratio.idxmax()
    return ans ,ratio.loc[ans]
answer_four()

('China', 0.689186772669207)


Question 5

Create a column that estimates the population using Energy Supply and Energy Supply per capita. What is the third most populous country according to this estimate?

In [10]:
def answer_five():
    Top15=answer_one()
    population=Top15['Energy Supply']/Top15['Energy Supply per Capita']
    Top15=Top15.assign(Population=population)
    ans_2=population.sort_values(ascending=False)
    ans_2=ans_2.index[2]

    return ans_2
answer_five()

'United States'

Question 6

Create a column that estimates the number of citable documents per person. What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the .corr() method, (Pearson's correlation).

In [14]:
def answer_six():
    Top15=answer_one()
    number=Top15['Citable documents']/(Top15['Energy Supply']/Top15['Energy Supply per Capita'])
    Top15=Top15.assign(Number=number)
    ans_3=number.corr(Top15['Energy Supply per Capita'])
    return ans_3
answer_six()

0.7272331912434825

Question 7

Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

In [16]:
def answer_seven():
    Top15=answer_one()
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    Top15['Population']=(Top15['Energy Supply']/Top15['Energy Supply per Capita'])
    Top15=Top15.reset_index()
    Top15['Continent']=Top15['Country']
    for country in Top15['Country']:
        Top15['Continent']=Top15['Country'].replace(ContinentDict)
        dff=Top15.set_index('Continent').groupby(level=0)['Population'].agg({'size':np.size,'sum':np.sum,'mean':np.mean,'std':np.std})
        ans_4=dff[['size', 'sum', 'mean', 'std']]
    return dff
answer_seven()

IndentationError: expected an indented block (291603856.py, line 22)